# Wind Field Tracking Analysis with ERA5 Data

This notebook demonstrates:
1. Downloading real ERA5 reanalysis wind data
2. Tracking wind advection velocity between measurement stations
3. Visualizing wind speed fields with velocity vectors

The tracking grid extends beyond the two measurement stations to properly capture wind field evolution.

## ⚙️ CDS API Setup (Required for Binder/Colab)

If you're running this notebook on Binder, Google Colab, or any environment without pre-configured CDS API credentials, run the cell below first.

**To get your API key:**
1. Register at [Copernicus Climate Data Store](https://cds.climate.copernicus.eu/)
2. Go to your profile page and copy your API key
3. Run the cell below and enter your credentials

In [ ]:
# CDS API Credential Setup (only needed for Binder/Colab)
# Skip this cell if you already have ~/.cdsapirc configured locally

import os
from pathlib import Path

def setup_cds_credentials():
    """Interactive setup for CDS API credentials."""
    cdsapirc_path = Path.home() / '.cdsapirc'
    
    # Check if credentials already exist
    if cdsapirc_path.exists():
        print("CDS API credentials already configured!")
        print(f"Location: {cdsapirc_path}")
        return True
    
    # Check if running in interactive environment
    try:
        api_key = input("Enter your CDS API key (format: UID:KEY, or press Enter to skip): ").strip()
        
        if not api_key:
            print("Skipped credential setup. You'll need to configure credentials before downloading data.")
            return False
        
        # Write credentials
        with open(cdsapirc_path, 'w') as f:
            f.write(f"url: https://cds.climate.copernicus.eu/api\n")
            f.write(f"key: {api_key}\n")
        
        print(f"Credentials saved to {cdsapirc_path}")
        return True
        
    except Exception as e:
        print(f"Could not set up credentials interactively: {e}")
        print("Please create ~/.cdsapirc manually with your credentials.")
        return False

# Uncomment the line below to run the setup
# setup_cds_credentials()

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime, timedelta
import cdsapi
import xarray as xr
from scipy.ndimage import gaussian_filter
from scipy.signal import correlate2d
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Define Measurement Stations and Extended Domain

We'll use two wind farms as example measurement stations:
- **Station 1 (Upstream)**: Horns Rev offshore wind farm, Denmark (55.5°N, 7.9°E)
- **Station 2 (Downstream)**: Anholt offshore wind farm, Denmark (56.6°N, 11.2°E)

The tracking grid extends 2° beyond the stations in all directions.

In [ ]:
# Define measurement station locations (wind farms)
station1 = {'name': 'Horns Rev', 'lat': 55.5, 'lon': 7.9}
station2 = {'name': 'Anholt', 'lat': 56.6, 'lon': 11.2}

# Calculate domain boundaries with extension
extension = 2.0  # degrees

lat_min = min(station1['lat'], station2['lat']) - extension
lat_max = max(station1['lat'], station2['lat']) + extension
lon_min = min(station1['lon'], station2['lon']) - extension
lon_max = max(station1['lon'], station2['lon']) + extension

print(f"Station 1 ({station1['name']}): {station1['lat']}°N, {station1['lon']}°E")
print(f"Station 2 ({station2['name']}): {station2['lat']}°N, {station2['lon']}°E")

# Calculate distance between stations
mean_lat = (station1['lat'] + station2['lat']) / 2
km_per_deg_lat = 111.0  # approximately constant
km_per_deg_lon = 111.0 * np.cos(np.deg2rad(mean_lat))

distance = np.sqrt(
    ((station2['lat'] - station1['lat']) * km_per_deg_lat)**2 +
    ((station2['lon'] - station1['lon']) * km_per_deg_lon)**2
)
print(f"\nDistance between stations: {distance:.1f} km")

print(f"\nExtended domain:")
print(f"  Latitude: {lat_min:.1f}°N to {lat_max:.1f}°N")
print(f"  Longitude: {lon_min:.1f}°E to {lon_max:.1f}°E")
print(f"  Extension: {extension}° beyond station boundaries")

## 2. Download ERA5 Data

Download 10m wind components (u, v) from ERA5 reanalysis using the CDS API.

In [ ]:
# Set up CDS API client
c = cdsapi.Client()

# Define date and time range for analysis
date_start = '2025-11-13'
date_end = '2025-11-14'
time_start = '00:00:00'
time_end = '00:00:00'

# Parse dates
from datetime import datetime
start_dt = datetime.strptime(f"{date_start} {time_start}", "%Y-%m-%d %H:%M:%S")
end_dt = datetime.strptime(f"{date_end} {time_end}", "%Y-%m-%d %H:%M:%S")

# Generate hourly time steps between start and end
times = []
current = start_dt
while current <= end_dt:
    times.append(current.strftime("%H:%M"))
    current += timedelta(hours=1)

# Get unique dates in the range
dates = []
current = start_dt
while current.date() <= end_dt.date():
    dates.append(current.strftime("%Y-%m-%d"))
    current += timedelta(days=1)

print(f"Downloading ERA5 data")
print(f"Date range: {date_start} to {date_end}")
print(f"Time range: {time_start} to {time_end}")
print(f"Total time steps: {len(times)}")
print(f"Domain: [{lat_min:.1f}, {lat_max:.1f}]°N, [{lon_min:.1f}, {lon_max:.1f}]°E")
print("\nThis may take a few minutes...")

In [ ]:
# Download ERA5 data
output_file = 'era5_wind_data.nc'

# Parse years, months, days from date range
years = list(set([d.split('-')[0] for d in dates]))
months = list(set([d.split('-')[1] for d in dates]))
days = list(set([d.split('-')[2] for d in dates]))

c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type': 'reanalysis',
        'variable': [
            '10m_u_component_of_wind',
            '10m_v_component_of_wind',
        ],
        'year': years,
        'month': months,
        'day': days,
        'time': times,
        'area': [lat_max, lon_min, lat_min, lon_max],  # North, West, South, East
        'format': 'netcdf',
    },
    output_file
)

print(f"\nData downloaded successfully to {output_file}")

## 3. Load and Process ERA5 Data

In [ ]:
# Load the downloaded data
ds = xr.open_dataset(output_file)

# ERA5 data from CDS API uses 'valid_time' instead of 'time' as the time dimension
# Rename it to 'time' for consistency
if 'valid_time' in ds.dims:
    ds = ds.rename({'valid_time': 'time'})

# Extract wind components
u10 = ds['u10']  # 10m u-component of wind
v10 = ds['v10']  # 10m v-component of wind

# Calculate wind speed
wind_speed = np.sqrt(u10**2 + v10**2)

# Add to dataset
ds['wind_speed'] = wind_speed

print("Dataset loaded successfully!")
print(f"\nTime steps: {len(ds.time)}")
print(f"Spatial dimensions: {len(ds.latitude)} x {len(ds.longitude)}")
print(f"\nVariables: {list(ds.data_vars)}")
print(f"\nTime range: {ds.time.values[0]} to {ds.time.values[-1]}")

In [ ]:
# Display dataset info
ds

## 4. Implement Wind Field Tracking

We use cross-correlation to track wind field patterns between consecutive time steps.
This gives us the advection velocity of wind patterns.

In [ ]:
def estimate_advection_velocity(field1, field2, dx, dy, dt, search_window=10):
    """
    Estimate advection velocity using cross-correlation between two wind fields.
    
    Parameters:
    -----------
    field1, field2 : 2D arrays
        Wind fields at consecutive times
    dx, dy : float
        Grid spacing in x and y (in km)
    dt : float
        Time difference (in hours)
    search_window : int
        Search radius in grid points
    
    Returns:
    --------
    vx, vy : float
        Advection velocity in x and y directions (m/s)
    """
    # Apply Gaussian smoothing to reduce noise
    field1_smooth = gaussian_filter(field1, sigma=1.5)
    field2_smooth = gaussian_filter(field2, sigma=1.5)
    
    # Compute cross-correlation
    correlation = correlate2d(field2_smooth, field1_smooth, mode='same')
    
    # Find peak correlation
    center = np.array(correlation.shape) // 2
    
    # Search in limited window around center
    search_slice = (
        slice(max(0, center[0]-search_window), min(correlation.shape[0], center[0]+search_window)),
        slice(max(0, center[1]-search_window), min(correlation.shape[1], center[1]+search_window))
    )
    
    search_region = correlation[search_slice]
    peak_idx = np.unravel_index(np.argmax(search_region), search_region.shape)
    
    # Convert to displacement from center
    peak_idx_global = (
        peak_idx[0] + max(0, center[0]-search_window),
        peak_idx[1] + max(0, center[1]-search_window)
    )
    
    displacement = np.array(peak_idx_global) - center
    
    # Convert to velocity (m/s)
    # displacement is in grid points, convert to km then m/s
    vx = (displacement[1] * dx * 1000) / (dt * 3600)  # m/s
    vy = (displacement[0] * dy * 1000) / (dt * 3600)  # m/s
    
    return vx, vy


def compute_optical_flow_velocity(field1, field2, dx, dy, dt, grid_spacing=5):
    """
    Compute optical flow-based velocity field using Lucas-Kanade method.
    
    Parameters:
    -----------
    field1, field2 : 2D arrays
        Wind fields at consecutive times
    dx, dy : float
        Grid spacing (in km)
    dt : float
        Time difference (in hours)
    grid_spacing : int
        Spacing for velocity grid points
    
    Returns:
    --------
    u_advect, v_advect : 2D arrays
        Advection velocity fields (m/s)
    x_grid, y_grid : 2D arrays
        Grid coordinates for velocity
    """
    # Compute spatial and temporal gradients
    fy, fx = np.gradient(field1)
    ft = field2 - field1
    
    # Initialize velocity fields
    ny, nx = field1.shape
    u_advect = np.zeros((ny // grid_spacing, nx // grid_spacing))
    v_advect = np.zeros((ny // grid_spacing, nx // grid_spacing))
    
    # Compute optical flow on coarse grid
    window = 5
    for i, y in enumerate(range(window, ny - window, grid_spacing)):
        for j, x in enumerate(range(window, nx - window, grid_spacing)):
            # Extract local window
            fx_win = fx[y-window:y+window+1, x-window:x+window+1].flatten()
            fy_win = fy[y-window:y+window+1, x-window:x+window+1].flatten()
            ft_win = ft[y-window:y+window+1, x-window:x+window+1].flatten()
            
            # Solve Lucas-Kanade equation: [fx, fy] * [u, v]^T = -ft
            A = np.column_stack([fx_win, fy_win])
            b = -ft_win
            
            # Least squares solution
            if np.linalg.matrix_rank(A.T @ A) == 2:
                velocity = np.linalg.lstsq(A, b, rcond=None)[0]
                
                # Convert from grid points to m/s
                u_advect[i, j] = velocity[0] * (dx * 1000) / (dt * 3600)
                v_advect[i, j] = velocity[1] * (dy * 1000) / (dt * 3600)
    
    # Create coordinate grids
    y_idx = np.arange(window, ny - window, grid_spacing)
    x_idx = np.arange(window, nx - window, grid_spacing)
    x_grid, y_grid = np.meshgrid(x_idx, y_idx)
    
    return u_advect, v_advect, x_grid, y_grid


print("Wind tracking functions defined successfully!")

## 5. Calculate Grid Spacing and Time Differences

In [ ]:
# Calculate grid spacing (approximate, in km)
lat = ds.latitude.values
lon = ds.longitude.values

# Grid spacing at mean latitude
mean_lat = np.mean(lat)
dlat = np.abs(np.diff(lat).mean())
dlon = np.abs(np.diff(lon).mean())

# Convert to km
km_per_deg_lat = 111.0  # approximately constant
km_per_deg_lon = 111.0 * np.cos(np.deg2rad(mean_lat))

dx = dlon * km_per_deg_lon  # km
dy = dlat * km_per_deg_lat  # km

# Time difference between consecutive steps
dt_hours = 1.0  # 1 hour between time steps

print(f"Grid spacing:")
print(f"  dx = {dx:.2f} km")
print(f"  dy = {dy:.2f} km")
print(f"  dt = {dt_hours} hour")

## 6. Track Wind Fields Between Time Steps

In [ ]:
# Compute advection velocities for all consecutive time pairs
n_times = len(ds.time)
advection_results = []

for i in range(n_times - 1):
    # Extract wind speed fields
    field1 = ds.wind_speed.isel(time=i).values
    field2 = ds.wind_speed.isel(time=i+1).values
    
    # Estimate bulk advection velocity
    vx, vy = estimate_advection_velocity(field1, field2, dx, dy, dt_hours)
    
    # Compute spatially varying optical flow
    u_advect, v_advect, x_grid, y_grid = compute_optical_flow_velocity(
        field1, field2, dx, dy, dt_hours, grid_spacing=8
    )
    
    advection_results.append({
        'time_pair': (i, i+1),
        'bulk_vx': vx,
        'bulk_vy': vy,
        'u_field': u_advect,
        'v_field': v_advect,
        'x_grid': x_grid,
        'y_grid': y_grid,
        'field1': field1,
        'field2': field2
    })
    
    time1 = ds.time.values[i]
    time2 = ds.time.values[i+1]
    print(f"Time {i}->{i+1} ({time1} to {time2}):")
    print(f"  Bulk advection velocity: vx = {vx:.2f} m/s, vy = {vy:.2f} m/s")
    print(f"  Speed: {np.sqrt(vx**2 + vy**2):.2f} m/s")
    print(f"  Direction: {np.rad2deg(np.arctan2(vy, vx)):.1f}° from East\n")

print(f"\nComputed advection for {len(advection_results)} time pairs")

## 7. Visualize Wind Speed Fields with Advection Vectors

Create maps showing:
- Wind speed as colored contours
- ERA5 wind vectors (from u10, v10)
- Tracked advection velocity as arrows
- Measurement station locations

In [ ]:
def plot_wind_field_with_tracking(ds, time_idx, advection_data=None, 
                                  station1=None, station2=None,
                                  figsize=(14, 10)):
    """
    Plot wind speed field with ERA5 wind vectors and tracking-derived advection.
    """
    fig = plt.figure(figsize=figsize)
    ax = plt.axes(projection=ccrs.PlateCarree())
    
    # Get data for this time
    wind_speed = ds.wind_speed.isel(time=time_idx)
    u10 = ds.u10.isel(time=time_idx)
    v10 = ds.v10.isel(time=time_idx)
    
    lon = ds.longitude.values
    lat = ds.latitude.values
    lon_grid, lat_grid = np.meshgrid(lon, lat)
    
    # Plot wind speed as filled contours
    # Convert max to scalar to avoid xarray wrapping issues with np.linspace
    levels = np.linspace(0, float(wind_speed.max().values), 20)
    contourf = ax.contourf(lon_grid, lat_grid, wind_speed.values,
                          levels=levels, cmap='YlOrRd', alpha=0.7,
                          transform=ccrs.PlateCarree())
    
    # Add colorbar
    cbar = plt.colorbar(contourf, ax=ax, orientation='vertical', 
                        pad=0.05, shrink=0.8)
    cbar.set_label('Wind Speed (m/s)', fontsize=12, fontweight='bold')
    
    # Plot ERA5 wind vectors (subsample for clarity)
    skip = 4
    quiver1 = ax.quiver(lon_grid[::skip, ::skip], lat_grid[::skip, ::skip],
                        u10.values[::skip, ::skip], v10.values[::skip, ::skip],
                        alpha=0.6, scale=200, width=0.003,
                        color='blue', label='ERA5 Wind',
                        transform=ccrs.PlateCarree())
    
    # Plot advection velocity vectors if provided
    if advection_data is not None:
        u_advect = advection_data['u_field']
        v_advect = advection_data['v_field']
        x_grid = advection_data['x_grid']
        y_grid = advection_data['y_grid']
        
        # Convert grid indices to lat/lon
        lon_advect = lon[x_grid]
        lat_advect = lat[y_grid]
        
        quiver2 = ax.quiver(lon_advect, lat_advect, u_advect, v_advect,
                           alpha=0.8, scale=200, width=0.004,
                           color='darkgreen', label='Tracked Advection',
                           transform=ccrs.PlateCarree())
    
    # Plot measurement stations
    if station1 is not None:
        ax.plot(station1['lon'], station1['lat'], 'r^', 
                markersize=15, markeredgecolor='black', markeredgewidth=2,
                label=f"Station 1: {station1['name']}",
                transform=ccrs.PlateCarree(), zorder=5)
    
    if station2 is not None:
        ax.plot(station2['lon'], station2['lat'], 'rv', 
                markersize=15, markeredgecolor='black', markeredgewidth=2,
                label=f"Station 2: {station2['name']}",
                transform=ccrs.PlateCarree(), zorder=5)
    
    # Add map features
    ax.coastlines(resolution='10m', linewidth=1.5)
    ax.add_feature(cfeature.BORDERS, linewidth=1)
    ax.add_feature(cfeature.LAND, alpha=0.3, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN, alpha=0.3, facecolor='lightblue')
    
    # Add gridlines
    gl = ax.gridlines(draw_labels=True, linewidth=0.5, 
                      color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    
    # Set extent
    ax.set_extent([lon.min(), lon.max(), lat.min(), lat.max()],
                  crs=ccrs.PlateCarree())
    
    # Add title with timestamp
    time_str = str(ds.time.values[time_idx])[:16]
    ax.set_title(f'Wind Speed Field and Advection Velocity\n{time_str} UTC',
                fontsize=14, fontweight='bold', pad=20)
    
    # Add legend
    ax.legend(loc='upper right', fontsize=10, framealpha=0.9)
    
    # Add quiver key
    ax.quiverkey(quiver1, 0.9, 0.95, 10, '10 m/s', 
                labelpos='E', coordinates='axes', color='blue')
    
    plt.tight_layout()
    return fig, ax

print("Visualization function defined!")

### 7.1 Plot Multiple Time Steps with Tracking

In [ ]:
# Plot first time step
fig, ax = plot_wind_field_with_tracking(
    ds, time_idx=0, 
    advection_data=advection_results[0],
    station1=station1, 
    station2=station2
)
plt.show()

In [ ]:
# Plot middle time step
mid_idx = len(ds.time) // 2
if mid_idx > 0 and mid_idx < len(advection_results):
    fig, ax = plot_wind_field_with_tracking(
        ds, time_idx=mid_idx,
        advection_data=advection_results[mid_idx-1],
        station1=station1,
        station2=station2
    )
    plt.show()

In [ ]:
# Plot last time step
if len(advection_results) > 0:
    fig, ax = plot_wind_field_with_tracking(
        ds, time_idx=-1,
        advection_data=advection_results[-1],
        station1=station1,
        station2=station2
    )
    plt.show()

### 7.2 Create Comparison: Wind Field Evolution

In [ ]:
# Create multi-panel plot showing evolution
fig = plt.figure(figsize=(18, 12))

# Compute global max for consistent color scale across panels
# Convert to scalar to avoid xarray wrapping issues with np.linspace
global_max = float(ds.wind_speed.max().values)

for idx in range(min(4, len(ds.time))):
    ax = plt.subplot(2, 2, idx+1, projection=ccrs.PlateCarree())
    
    # Get data
    wind_speed = ds.wind_speed.isel(time=idx)
    u10 = ds.u10.isel(time=idx)
    v10 = ds.v10.isel(time=idx)
    
    lon = ds.longitude.values
    lat = ds.latitude.values
    lon_grid, lat_grid = np.meshgrid(lon, lat)
    
    # Plot wind speed
    levels = np.linspace(0, global_max, 20)
    contourf = ax.contourf(lon_grid, lat_grid, wind_speed.values,
                          levels=levels, cmap='YlOrRd', alpha=0.7,
                          transform=ccrs.PlateCarree())
    
    # Plot wind vectors
    skip = 4
    ax.quiver(lon_grid[::skip, ::skip], lat_grid[::skip, ::skip],
             u10.values[::skip, ::skip], v10.values[::skip, ::skip],
             alpha=0.6, scale=200, width=0.003, color='blue',
             transform=ccrs.PlateCarree())
    
    # Plot advection if available
    if idx < len(advection_results):
        adv = advection_results[idx]
        lon_advect = lon[adv['x_grid']]
        lat_advect = lat[adv['y_grid']]
        
        ax.quiver(lon_advect, lat_advect, adv['u_field'], adv['v_field'],
                 alpha=0.8, scale=200, width=0.004, color='darkgreen',
                 transform=ccrs.PlateCarree())
    
    # Plot stations
    ax.plot(station1['lon'], station1['lat'], 'r^', markersize=12,
           markeredgecolor='black', markeredgewidth=1.5,
           transform=ccrs.PlateCarree(), zorder=5)
    ax.plot(station2['lon'], station2['lat'], 'rv', markersize=12,
           markeredgecolor='black', markeredgewidth=1.5,
           transform=ccrs.PlateCarree(), zorder=5)
    
    # Add features
    ax.coastlines(resolution='10m', linewidth=1)
    ax.add_feature(cfeature.LAND, alpha=0.3, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN, alpha=0.3, facecolor='lightblue')
    
    # Gridlines
    gl = ax.gridlines(draw_labels=True, linewidth=0.5, 
                     color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    if idx % 2 == 1:
        gl.left_labels = False
    if idx < 2:
        gl.bottom_labels = False
    
    # Title
    time_str = str(ds.time.values[idx])[:16]
    ax.set_title(f'{time_str} UTC', fontsize=12, fontweight='bold')
    
    # Set extent
    ax.set_extent([lon.min(), lon.max(), lat.min(), lat.max()],
                 crs=ccrs.PlateCarree())

# Add colorbar
cbar = plt.colorbar(contourf, ax=fig.get_axes(), 
                   orientation='horizontal', pad=0.05, shrink=0.8)
cbar.set_label('Wind Speed (m/s)', fontsize=12, fontweight='bold')

fig.suptitle('Wind Field Evolution with Tracked Advection Velocity',
            fontsize=16, fontweight='bold', y=0.98)

plt.tight_layout()
plt.show()

## 8. Detailed Advection Analysis

In [ ]:
# Plot advection velocity magnitude and direction
if len(advection_results) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6),
                            subplot_kw={'projection': ccrs.PlateCarree()})
    
    # Use middle time step
    adv = advection_results[len(advection_results)//2]
    
    lon = ds.longitude.values
    lat = ds.latitude.values
    
    lon_advect = lon[adv['x_grid']]
    lat_advect = lat[adv['y_grid']]
    
    # Calculate magnitude and direction
    magnitude = np.sqrt(adv['u_field']**2 + adv['v_field']**2)
    direction = np.rad2deg(np.arctan2(adv['v_field'], adv['u_field']))
    
    # Plot 1: Magnitude
    ax = axes[0]
    contourf = ax.contourf(lon_advect, lat_advect, magnitude,
                          levels=15, cmap='viridis', alpha=0.8,
                          transform=ccrs.PlateCarree())
    
    ax.quiver(lon_advect, lat_advect, adv['u_field'], adv['v_field'],
             alpha=0.6, scale=150, width=0.004,
             transform=ccrs.PlateCarree())
    
    ax.plot(station1['lon'], station1['lat'], 'r^', markersize=12,
           markeredgecolor='white', markeredgewidth=2,
           transform=ccrs.PlateCarree(), zorder=5)
    ax.plot(station2['lon'], station2['lat'], 'rv', markersize=12,
           markeredgecolor='white', markeredgewidth=2,
           transform=ccrs.PlateCarree(), zorder=5)
    
    ax.coastlines(resolution='10m')
    ax.add_feature(cfeature.LAND, alpha=0.3)
    ax.gridlines(draw_labels=True, alpha=0.5)
    ax.set_title('Advection Velocity Magnitude', fontsize=13, fontweight='bold')
    
    cbar = plt.colorbar(contourf, ax=ax, orientation='horizontal', pad=0.05)
    cbar.set_label('Speed (m/s)', fontsize=11)
    
    # Plot 2: Direction
    ax = axes[1]
    contourf2 = ax.contourf(lon_advect, lat_advect, direction,
                           levels=np.linspace(-180, 180, 21), 
                           cmap='twilight', alpha=0.8,
                           transform=ccrs.PlateCarree())
    
    ax.quiver(lon_advect, lat_advect, adv['u_field'], adv['v_field'],
             alpha=0.6, scale=150, width=0.004,
             transform=ccrs.PlateCarree())
    
    ax.plot(station1['lon'], station1['lat'], 'r^', markersize=12,
           markeredgecolor='white', markeredgewidth=2,
           transform=ccrs.PlateCarree(), zorder=5)
    ax.plot(station2['lon'], station2['lat'], 'rv', markersize=12,
           markeredgecolor='white', markeredgewidth=2,
           transform=ccrs.PlateCarree(), zorder=5)
    
    ax.coastlines(resolution='10m')
    ax.add_feature(cfeature.LAND, alpha=0.3)
    ax.gridlines(draw_labels=True, alpha=0.5)
    ax.set_title('Advection Velocity Direction', fontsize=13, fontweight='bold')
    
    cbar2 = plt.colorbar(contourf2, ax=ax, orientation='horizontal', pad=0.05)
    cbar2.set_label('Direction (° from East)', fontsize=11)
    
    plt.tight_layout()
    plt.show()

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def create_wind_animation(ds, advection_results, station1, station2, figsize=(14, 10)):
    """
    Create an animated map showing wind speed and advection velocity.
    
    Parameters:
    -----------
    ds : xarray.Dataset
        Dataset containing wind data
    advection_results : list
        List of advection tracking results
    station1, station2 : dict
        Station location dictionaries
    figsize : tuple
        Figure size
    
    Returns:
    --------
    animation : FuncAnimation
        Matplotlib animation object
    """
    fig = plt.figure(figsize=figsize)
    ax = plt.axes(projection=ccrs.PlateCarree())
    
    lon = ds.longitude.values
    lat = ds.latitude.values
    lon_grid, lat_grid = np.meshgrid(lon, lat)
    
    # Compute global max for consistent color scale
    global_max = float(ds.wind_speed.max().values)
    levels = np.linspace(0, global_max, 20)
    
    # Initialize empty plot elements
    contourf_plot = None
    quiver_plot = None
    title_text = None
    
    def init():
        """Initialize animation."""
        ax.coastlines(resolution='10m', linewidth=1.5)
        ax.add_feature(cfeature.BORDERS, linewidth=1)
        ax.add_feature(cfeature.LAND, alpha=0.3, facecolor='lightgray')
        ax.add_feature(cfeature.OCEAN, alpha=0.3, facecolor='lightblue')
        
        # Add gridlines
        gl = ax.gridlines(draw_labels=True, linewidth=0.5, 
                          color='gray', alpha=0.5, linestyle='--')
        gl.top_labels = False
        gl.right_labels = False
        
        # Set extent
        ax.set_extent([lon.min(), lon.max(), lat.min(), lat.max()],
                      crs=ccrs.PlateCarree())
        
        # Plot stations (static)
        ax.plot(station1['lon'], station1['lat'], 'r^', 
                markersize=15, markeredgecolor='black', markeredgewidth=2,
                label=f"Station 1: {station1['name']}",
                transform=ccrs.PlateCarree(), zorder=5)
        ax.plot(station2['lon'], station2['lat'], 'rv', 
                markersize=15, markeredgecolor='black', markeredgewidth=2,
                label=f"Station 2: {station2['name']}",
                transform=ccrs.PlateCarree(), zorder=5)
        
        ax.legend(loc='upper right', fontsize=10, framealpha=0.9)
        
        return []
    
    def update(frame):
        """Update animation frame."""
        nonlocal contourf_plot, quiver_plot, title_text
        
        # Clear previous contourf and quiver
        if contourf_plot is not None:
            for coll in contourf_plot.collections:
                coll.remove()
        if quiver_plot is not None:
            quiver_plot.remove()
        if title_text is not None:
            title_text.remove()
        
        # Get wind speed for this time
        wind_speed = ds.wind_speed.isel(time=frame)
        
        # Plot wind speed as filled contours
        contourf_plot = ax.contourf(lon_grid, lat_grid, wind_speed.values,
                                     levels=levels, cmap='YlOrRd', alpha=0.7,
                                     transform=ccrs.PlateCarree())
        
        # Overlay advection velocity if available
        # Use optical flow advection (spatially varying)
        if frame < len(advection_results):
            adv = advection_results[frame]
            u_advect = adv['u_field']
            v_advect = adv['v_field']
            x_grid = adv['x_grid']
            y_grid = adv['y_grid']
            
            # Convert grid indices to lat/lon
            lon_advect = lon[x_grid]
            lat_advect = lat[y_grid]
            
            # Plot advection velocity vectors
            quiver_plot = ax.quiver(lon_advect, lat_advect, u_advect, v_advect,
                                    alpha=0.8, scale=200, width=0.004,
                                    color='darkgreen', 
                                    transform=ccrs.PlateCarree(), zorder=4)
        
        # Update title with timestamp
        time_str = str(ds.time.values[frame])[:16]
        title_text = ax.set_title(
            f'Wind Speed and Advection Velocity\\n{time_str} UTC',
            fontsize=14, fontweight='bold', pad=20
        )
        
        return []
    
    # Create animation
    n_frames = len(ds.time)
    anim = FuncAnimation(fig, update, init_func=init, frames=n_frames,
                        interval=500, blit=False, repeat=True)
    
    return anim

# Create the animation
if len(advection_results) > 0:
    print("Creating animation...")
    anim = create_wind_animation(ds, advection_results, station1, station2)
    
    # Display animation in notebook
    # Note: In Jupyter notebooks, this will show an interactive animation
    # In some environments, you may need to save to file instead
    plt.close()  # Prevent static display
    HTML(anim.to_jshtml())
else:
    print("No advection results available for animation.")

## 8.1 Animated Wind Speed with Advection Velocity

Create an animated map showing:
- Wind speed evolution per hour as colored fields
- Advection velocity vectors overlaid on top (from optical flow method)

## 9. Time Series Analysis at Stations

In [ ]:
# Extract time series at station locations
def extract_at_location(ds, lat, lon):
    """Extract data at nearest grid point to given location."""
    return ds.sel(latitude=lat, longitude=lon, method='nearest')

# Get data at stations
station1_data = extract_at_location(ds, station1['lat'], station1['lon'])
station2_data = extract_at_location(ds, station2['lat'], station2['lon'])

# Create time series plot
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

times = ds.time.values

# Wind speed
ax = axes[0]
ax.plot(times, station1_data.wind_speed.values, 'o-', 
        linewidth=2, markersize=8, label=station1['name'], color='red')
ax.plot(times, station2_data.wind_speed.values, 's-', 
        linewidth=2, markersize=8, label=station2['name'], color='blue')
ax.set_ylabel('Wind Speed (m/s)', fontsize=12, fontweight='bold')
ax.legend(fontsize=11, loc='best')
ax.grid(True, alpha=0.3)
ax.set_title('Wind Speed Time Series at Stations', fontsize=13, fontweight='bold')

# U component
ax = axes[1]
ax.plot(times, station1_data.u10.values, 'o-', 
        linewidth=2, markersize=8, label=station1['name'], color='red')
ax.plot(times, station2_data.u10.values, 's-', 
        linewidth=2, markersize=8, label=station2['name'], color='blue')
ax.axhline(0, color='black', linewidth=0.5, linestyle='--')
ax.set_ylabel('U Component (m/s)', fontsize=12, fontweight='bold')
ax.legend(fontsize=11, loc='best')
ax.grid(True, alpha=0.3)

# V component
ax = axes[2]
ax.plot(times, station1_data.v10.values, 'o-', 
        linewidth=2, markersize=8, label=station1['name'], color='red')
ax.plot(times, station2_data.v10.values, 's-', 
        linewidth=2, markersize=8, label=station2['name'], color='blue')
ax.axhline(0, color='black', linewidth=0.5, linestyle='--')
ax.set_ylabel('V Component (m/s)', fontsize=12, fontweight='bold')
ax.set_xlabel('Time (UTC)', fontsize=12, fontweight='bold')
ax.legend(fontsize=11, loc='best')
ax.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 10. Summary Statistics

In [ ]:
# Compute summary statistics
print("=" * 70)
print("WIND FIELD TRACKING ANALYSIS SUMMARY")
print("=" * 70)
print(f"\nAnalysis Period: {ds.time.values[0]} to {ds.time.values[-1]}")
print(f"Number of Time Steps: {len(ds.time)}")
print(f"\nDomain:")
print(f"  Latitude:  {lat_min:.2f}°N to {lat_max:.2f}°N")
print(f"  Longitude: {lon_min:.2f}°E to {lon_max:.2f}°E")
print(f"  Grid Size: {len(ds.latitude)} x {len(ds.longitude)} points")

print(f"\nStation Locations:")
print(f"  {station1['name']:15s}: {station1['lat']:.2f}°N, {station1['lon']:.2f}°E")
print(f"  {station2['name']:15s}: {station2['lat']:.2f}°N, {station2['lon']:.2f}°E")

distance = np.sqrt(
    ((station2['lat'] - station1['lat']) * km_per_deg_lat)**2 +
    ((station2['lon'] - station1['lon']) * km_per_deg_lon)**2
)
print(f"  Distance between stations: {distance:.1f} km")

print(f"\nWind Speed Statistics:")
print(f"  Domain average: {ds.wind_speed.mean().values:.2f} m/s")
print(f"  Domain maximum: {ds.wind_speed.max().values:.2f} m/s")
print(f"  Domain minimum: {ds.wind_speed.min().values:.2f} m/s")
print(f"  Domain std dev: {ds.wind_speed.std().values:.2f} m/s")

print(f"\nAdvection Velocity Statistics:")
if len(advection_results) > 0:
    bulk_vx = [r['bulk_vx'] for r in advection_results]
    bulk_vy = [r['bulk_vy'] for r in advection_results]
    bulk_speed = [np.sqrt(vx**2 + vy**2) for vx, vy in zip(bulk_vx, bulk_vy)]
    
    print(f"  Mean advection speed: {np.mean(bulk_speed):.2f} m/s")
    print(f"  Max advection speed:  {np.max(bulk_speed):.2f} m/s")
    print(f"  Mean vx (eastward):   {np.mean(bulk_vx):.2f} m/s")
    print(f"  Mean vy (northward):  {np.mean(bulk_vy):.2f} m/s")

print(f"\nStation Wind Speeds:")
print(f"  {station1['name']} average: {station1_data.wind_speed.mean().values:.2f} m/s")
print(f"  {station2['name']} average: {station2_data.wind_speed.mean().values:.2f} m/s")

print("\n" + "=" * 70)

## 11. Save Results

In [ ]:
# Save tracking results to file
import pickle

results_dict = {
    'advection_results': advection_results,
    'station1': station1,
    'station2': station2,
    'domain': {
        'lat_min': lat_min,
        'lat_max': lat_max,
        'lon_min': lon_min,
        'lon_max': lon_max
    },
    'grid_spacing': {'dx': dx, 'dy': dy, 'dt': dt_hours}
}

with open('tracking_results.pkl', 'wb') as f:
    pickle.dump(results_dict, f)

print("Results saved to tracking_results.pkl")

# Also save as NetCDF for further analysis
ds.to_netcdf('processed_era5_data.nc')
print("Processed data saved to processed_era5_data.nc")

## Conclusion

This notebook has demonstrated:

1. **ERA5 Data Acquisition**: Downloaded real wind data from ERA5 reanalysis
2. **Extended Domain**: Created a grid extending beyond the two measurement stations
3. **Wind Tracking**: Implemented both bulk and spatially-varying advection velocity estimation
4. **Comprehensive Visualization**: Created multiple maps showing:
   - Wind speed fields as contours
   - ERA5 wind vectors
   - Tracked advection velocity as arrows
   - Station locations
   - Time evolution

The tracking algorithms allow us to quantify how wind patterns move across the domain,
which is valuable for understanding wind field evolution and predicting conditions at
downstream locations.